In [ ]:
 !pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu116


In [ ]:
!pip install datasets==1.0.2
!pip install tqdm==4.57.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!mkdir '/content/drive/My Drive/t5'
!mkdir '/content/drive/My Drive/t5/dataset'
!mkdir '/content/drive/My Drive/t5/model'
!mkdir '/content/drive/My Drive/t5/tokenizer'
!ls

mkdir: cannot create directory ‘/content/drive/My Drive/t5’: File exists
mkdir: cannot create directory ‘/content/drive/My Drive/t5/dataset’: File exists
mkdir: cannot create directory ‘/content/drive/My Drive/t5/model’: File exists
mkdir: cannot create directory ‘/content/drive/My Drive/t5/tokenizer’: File exists
drive  lightning_logs  sample_data


In [ ]:
from datasets import load_dataset

In [ ]:
import pandas as pd

## 2. Train T5 transformer model

In [ ]:
!pip install --quiet transformers==4.1.1
!pip install --quiet tokenizers==0.9.4 
!pip install --quiet sentencepiece==0.1.94
!pip install --quiet tqdm==4.57.0
!pip install --quiet pytorch-lightning==1.2.10
!pip install --quiet torchtext==0.10.0

# !pip install --quiet transformers==4.1.1
# !pip install --quiet pytorch-lightning==1.1.3
# !pip install --quiet tokenizers==0.9.4 
# !pip install --quiet sentencepiece==0.1.94
# !pip install --quiet tqdm==4.56.0
# !pip install --quiet torchtext==0.8.1

In [ ]:
!pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [ ]:
# restart runtime
import os

def restart_runtime():
  os.kill(os.getpid(), 9)

restart_runtime()

In [ ]:
# train_file_path = '/content/drive/My Drive/t5/dataset/squad_t5_train.csv'
# validation_file_path = '/content/drive/My Drive/t5/dataset/squad_t5_val.csv'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Check we have a GPU and check the memory size of the GUP
!nvidia-smi

Wed Feb 22 14:58:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    27W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**Import packages**

In [ ]:
!pip install pytorch_lightning
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import time
import random

import argparse
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

from transformers import T5ForConditionalGeneration,T5Tokenizer

pl.seed_everything(42)

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


42

In [ ]:
from datasets import load_dataset

# Download the SQuAD dataset
dataset = load_dataset('squad')
print(dataset)
print()
# Print the first example in the training set
print(dataset['train'][0])
print()
# Access the context and question of the first example
context = dataset['train'][0]['context']
question = dataset['train'][0]['question']

# Print the context and question
print('Context:', context)
print()
print('Question:', question)


Reusing dataset squad (/root/.cache/huggingface/datasets/squad/plain_text/1.0.0/1244d044b266a5e4dbd4174d23cb995eead372fbca31a03edc3f8a132787af41)


DatasetDict({'train': Dataset(features: {'id': Value(dtype='string', id=None), 'title': Value(dtype='string', id=None), 'context': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}, num_rows: 87599), 'validation': Dataset(features: {'id': Value(dtype='string', id=None), 'title': Value(dtype='string', id=None), 'context': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}, num_rows: 10570)})

{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']}, 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is 

In [ ]:
train_data = dataset['train']
train_data

Dataset(features: {'id': Value(dtype='string', id=None), 'title': Value(dtype='string', id=None), 'context': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}, num_rows: 87599)

In [ ]:
len(train_data)

87599

In [ ]:
print(train_data[0])

{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']}, 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'id': '5733be284776f41900661182', 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'title': 'University_of_Notre_Dame'}


In [ ]:
print(train_data[0]['context'])
print(train_data[0]['question'])
print(train_data[0]['answers']['text'][0])

Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Saint Bernadette Soubirous


In [ ]:
print(train_data[99]['context'])


One of the main driving forces in the growth of the University was its football team, the Notre Dame Fighting Irish. Knute Rockne became head coach in 1918. Under Rockne, the Irish would post a record of 105 wins, 12 losses, and five ties. During his 13 years the Irish won three national championships, had five undefeated seasons, won the Rose Bowl in 1925, and produced players such as George Gipp and the "Four Horsemen". Knute Rockne has the highest winning percentage (.881) in NCAA Division I/FBS football history. Rockne's offenses employed the Notre Dame Box and his defenses ran a 7–2–2 scheme. The last game Rockne coached was on December 14, 1930 when he led a group of Notre Dame all-stars against the New York Giants in New York City.


In [ ]:
%%time
context = []
question = []
answer = []

for i in range(len(train_data)):

    context.append(train_data[i]['context'])
    question.append(train_data[i]['question'])
    answer.append(train_data[i]['answers']['text'][0])

CPU times: user 14.2 s, sys: 117 ms, total: 14.3 s
Wall time: 14.5 s


In [ ]:
print(len(context))
print(len(question))
print(len(answer))

87599
87599
87599


In [ ]:
df = pd.DataFrame(list(zip(context,question,answer)), columns=['context','question','answer'])
print(df.shape)

(87599, 3)


In [ ]:
df = df[:4000]

In [ ]:
df.sample(10)

,context,question,answer
555,Beyoncé embarked on The Mrs. Carter Show World...,Of what event was Beyonce honorary chair?,2013 Met Gala
3491,The earthquake left at least 5 million people ...,"Because of the earthquake, how many people did...",at least 5 million
527,"On January 7, 2012, Beyoncé gave birth to her ...",Beyonce's first child is named what?,Blue Ivy Carter
3925,The only attempt at a peaceful solution to the...,Which natural disaster happened after English ...,the Great Fire of New York
2989,"On release, Twilight Princess was considered t...",Which game journalist was perturbed by a lack ...,Javier Glickman
70,As of 2012[update] research continued in many ...,What does the Kroc Institute at Notre Dame foc...,International Peace studies
1756,"In the spring of 1834, Chopin attended the Low...",Who did Frédéric meet in the spring of 1834 at...,Felix Mendelssohn
211,"The television station, NDtv, grew from one sh...",How many programs did NDtv feature in 2002?,one show
196,"Besides its prominence in sports, Notre Dame i...",How many individual colleges are part of Notre...,four
803,"In September 2010, Beyoncé made her runway mod...",Which month and year did GQ feature Beyonce on...,January 2013


In [ ]:
maxm = 0
for i in range(df.shape[0]):
  txt = df['context'][i] + ' ' + df['question'][i]
  total_len = len(txt.split())
  if total_len > maxm:
     maxm = total_len
maxm

512

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split

test, valid = train_test_split(test, test_size=0.2, random_state=42)

In [ ]:
train.reset_index(drop=True,inplace=True)
valid.reset_index(drop=True,inplace=True)
test.reset_index(drop=True,inplace=True)

In [ ]:
print(df.shape)
print(train.shape)
print(valid.shape)
print(test.shape)

(4000, 3)
(3200, 3)
(160, 3)
(640, 3)


In [ ]:
train.head(3)

,context,question,answer
0,"Beyoncé Giselle Knowles was born in Houston, T...",What company did Beyoncé's father work for whe...,Xerox
1,"Kolmaš writes that, as the Mongol presence in ...",What were the Mongols trying to reclaim?,their old vassal of Tibet
2,"Foxconn, Apple's manufacturer, initially denie...",What auditor company did Apple hire to oversee...,Verité


**Preparing Dataset**

In [ ]:
# inheriting 2 classes from pytorch ==> dataset, t5tuner
# we are directly using t5tuner class
# but, we are not directly using dataset class,
# we are taking help of dataset class to generate data which is compatible with pytorch and which
# can be used in t5tuner.

# T5 from hugging face is pytorch based, so in that case if data is not pytorch compatible,
# it will have difficulty in generating graph, which inturn is helpful in back propagation.

In [ ]:
from tqdm.notebook import tqdm
import copy

class AnswerGenerationDataset(Dataset):
    def __init__(self, tokenizer, dataframe, max_len_inp=512,max_len_out=100):

        self.passage_column = "context"
        self.answer = "answer"
        self.question = "question"

        # self.data = pd.read_csv(self.path)
        self.data = dataframe.copy()

        

        self.max_len_input = max_len_inp
        self.max_len_output = max_len_out
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []
        self.skippedcount =0
        self._build()
    def __len__(self):
        return len(self.inputs)
    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        src_mask = self.inputs[index]["attention_mask"].squeeze()  # might need to squeeze
        target_mask = self.targets[index]["attention_mask"].squeeze()  # might need to squeeze

        labels = copy.deepcopy(target_ids)
        labels [labels==0] = -100

        return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask,"labels":labels}
    def _build(self):
        for idx in tqdm(range(len(self.data))):

            question,context,target = self.data.loc[idx, self.question],self.data.loc[idx, self.passage_column], self.data.loc[idx, self.answer]

            input_ = f'question: {question} \n\n context: {context} </s>'
            target = " %s </s>" % (str(target))
            #input = f'Summarization : {paragraph} </s>"
            # get encoding length of input. If it is greater than self.max_len skip it
            test_input_encoding = self.tokenizer.encode_plus(input_,
                                        truncation=True,
                                        return_tensors="pt")
            
            length_of_input_encoding = len(test_input_encoding['input_ids'][0])

            tokenized_inputs = self.tokenizer.batch_encode_plus(
                [input_], max_length=self.max_len_input, pad_to_max_length=True, return_tensors="pt",truncation=True
            )
            # tokenize targets
            tokenized_targets = self.tokenizer.batch_encode_plus(
                [target], max_length=self.max_len_output, pad_to_max_length=True,return_tensors="pt",truncation=True
            )
            self.inputs.append(tokenized_inputs)
            self.targets.append(tokenized_targets)


**T5 Finetuner**

In [ ]:
class T5FineTuner(pl.LightningModule):
    def __init__(self,hparams, t5model, t5tokenizer,train_dataset,validation_dataset):
        super(T5FineTuner, self).__init__()
        self.hparams = hparams
        self.model = t5model
        # self.tokenizer = t5tokenizer
        self.validation_dataset = validation_dataset
        self.train_dataset = train_dataset

    def forward( self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, lm_labels=None):
         outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=decoder_attention_mask,
            labels=lm_labels,
        )
         return outputs


    def training_step(self, batch, batch_idx):
        outputs = self.forward(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_input_ids = batch["target_ids"],
            decoder_attention_mask=batch['target_mask'],
            lm_labels=batch['labels']
        )
        loss = outputs[0]
        self.log('train_loss',loss)
        return loss

    def validation_step(self, batch, batch_idx):
        outputs = self.forward(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_input_ids = batch["target_ids"],
            decoder_attention_mask=batch['target_mask'],
            lm_labels=batch['labels']
        )
        loss = outputs[0]
        self.log("val_loss",loss)
        return loss

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.hparams.batch_size,num_workers=4)

  
    def val_dataloader(self):
        return DataLoader(self.validation_dataset, batch_size=self.hparams.batch_size,num_workers=4)



    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=3e-4, eps=1e-8)
        return optimizer




In [ ]:
def dataset_creation_and_training(saved_model_path,saved_trained_tokenizer,train_dataframe,test_dataframe,t5_tokenizer,t5_model,epochs):
      
      print(f'Preparing the Train_Dataset')
      train_dataset = AnswerGenerationDataset(t5_tokenizer,train_dataframe)
      print('training dataset generation completed')
      print(f'Preparing the Validation_Dataset')
      validation_dataset = AnswerGenerationDataset(t5_tokenizer,test_dataframe)

      args_dict = dict(
          batch_size=4,
      )

      args = argparse.Namespace(**args_dict)


      model = T5FineTuner(args,t5_model,t5_tokenizer,train_dataset,validation_dataset)

      trainer = pl.Trainer(max_epochs = epochs, gpus=1,progress_bar_refresh_rate=30)
      #trainer = pl.Trainer(max_epochs = 1,progress_bar_refresh_rate=30)


      trainer.fit(model)

      print ("Saving model")
      save_path_model = saved_model_path
      save_path_tokenizer = saved_trained_tokenizer
      model.model.save_pretrained(save_path_model)
      t5_tokenizer.save_pretrained(save_path_tokenizer)

In [ ]:
def generate_answers(text,tokenizer,model,device):

    
    encoding = tokenizer.encode_plus(text,max_length =512, padding=True, return_tensors="pt")
    input_ids,attention_mask  = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

    model.eval()
    beam_outputs = model.generate(
        input_ids=input_ids,attention_mask=attention_mask,
        max_length=72,
        early_stopping=True,
        num_beams=5,
        num_return_sequences=3

    )
    #print(beam_outputs)
    answer = tokenizer.decode(beam_outputs[0], skip_special_tokens=True,clean_up_tokenization_spaces=True)

    return answer

In [ ]:
def preprocess_test_df(df):
    # columns ==> answer, question, context
    df['text'] = "question: "+ df['question'] + ' ' + "\n\n"+ ' ' + "context: " + df['context'] + " </s>"
    # columns ==> text, answer, question, context
    return df

In [ ]:
def predict_answers(df ,tokenizer,model,device):

  df = preprocess_test_df(df)
  answers = []
  
  for i in range(df.shape[0]):
      text = df.loc[i,'text']
      original_answer = df.loc[i,'answer']
      question = df.loc[i,'question']
      new_answer = generate_answers(text ,tokenizer,model,device)
      ans = answers.append(new_answer)
      # text, answer, question , context, generated_answer

  df['Generated_answer'] = np.array(answers)

  
  return df

In [ ]:
def training_schedule(saved_model_path,saved_trained_tokenizer,train_dataframe,test_dataframe,t5_tokenizer,t5_model,epochs):

  dataset_creation_and_training(saved_model_path,saved_trained_tokenizer,train_dataframe,test_dataframe,t5_tokenizer,t5_model,epochs)


In [ ]:
def predicting_schedule(test_df,saved_model_path,saved_trained_tokenizer,model_name,df_type,device):

  model = T5ForConditionalGeneration.from_pretrained(saved_model_path)
  tokenizer = T5Tokenizer.from_pretrained(saved_trained_tokenizer)

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print ("device ",device)
  model = model.to(device)

  df_answer = predict_answers(test_df ,tokenizer,model,device)

  return df_answer

In [ ]:
# model_list = ["t5-small","t5-base"]



In [ ]:
%%time
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelWithLMHead

saved_model_path = '/content/drive/My Drive/t5/model/'
saved_trained_tokenizer = '/content/drive/My Drive/t5/tokenizer/'

epochs = 2
df_type = 'predicted_answer'

model_name = "t5-base"

t5_tokenizer = AutoTokenizer.from_pretrained(model_name)

t5_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

training_schedule(saved_model_path,saved_trained_tokenizer,train,valid,t5_tokenizer,t5_model,epochs)



Some weights of the model checkpoint at t5-base were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Preparing the Train_Dataset


  0%|          | 0/3200 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2173: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


training dataset generation completed
Preparing the Validation_Dataset


...

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Saving model
CPU times: user 16min 28s, sys: 1min 14s, total: 17min 42s
Wall time: 18min 15s


In [ ]:
df_answer = predicting_schedule(test,saved_model_path,saved_trained_tokenizer,model_name,df_type,device)
df_answer


device  cuda


/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5.py:183: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


,context,question,answer,text,Generated_answer
0,"Following filming in Mexico, and during a sche...",When did Craig go back to work?,22 April,question: When did Craig go back to work? \n\n...,22 April
1,Beyoncé has received praise for her stage pres...,For what does Beyonce receive praise?,stage presence,question: For what does Beyonce receive praise...,her stage presence and voice
2,"In Paris, Chopin encountered artists and other...",What did Frédéric create from verses of the po...,songs,question: What did Frédéric create from verses...,the Polish Literary Society
3,About 80% of undergraduates and 20% of graduat...,How many student housing areas are reserved fo...,four,question: How many student housing areas are r...,four
4,"Simultaneously with these conflicts, bison, a ...",In 1884 about how many bison remained?,about 325,question: In 1884 about how many bison remaine...,325
...,...,...,...,...,...
635,Average annual precipitation is 15 inches (380...,How much precipitation does the Grinnell Glac...,105 inches,question: How much precipitation does the Gri...,105 inches
636,"In the spring of 1834, Chopin attended the Low...",In 1835 where did Chopin and his parents visit?,Carlsbad,question: In 1835 where did Chopin and his par...,Carlsbad
637,All of Chopin's compositions include the piano...,Which instrument do every one of his compositi...,piano,question: Which instrument do every one of his...,piano
638,"In the film Knute Rockne, All American, Knute ...",Pat O'Brien portrayed which person in the film...,Knute Rockne,question: Pat O'Brien portrayed which person i...,All American


In [ ]:
df_answer.sample(20)

,context,question,answer,text,Generated_answer
439,Apple's application to the United States Paten...,Which rival company sued Apple in 2005 and 2006?,Creative Technology,question: Which rival company sued Apple in 20...,Creative Technology
171,"Two of Chopin's long-standing pupils, Karol Mi...",Who was a student of Chopin's former students ...,Raoul Koczalski,question: Who was a student of Chopin's former...,Raoul Koczalski
246,"Antibacterial-resistant strains and species, s...",What was a once almost controlled disease that...,tuberculosis,question: What was a once almost controlled di...,tuberculosis
451,At the 57th Annual Grammy Awards in February 2...,Beyonce would perform with who at Superbowl 50?,Coldplay,question: Beyonce would perform with who at Su...,Coldplay
589,"From 1842 onwards, Chopin showed signs of seri...",What specific illness does modern evidence poi...,temporal lobe epilepsy,question: What specific illness does modern ev...,temporal lobe epilepsy
529,"Father Joseph Carrier, C.S.C. was Director of ...",What was the lifespan of John Augustine Zahm?,1851–1921,question: What was the lifespan of John August...,1851–1921
59,The effects of some types of mold on infection...,Who noticed in a lab the antibacterial charact...,Alexander Fleming,question: Who noticed in a lab the antibacteri...,Alexander Fleming
71,Strong aftershocks continued to strike even mo...,How many homes were destroyed because of the M...,"more than 420,000",question: How many homes were destroyed becaus...,"420,000"
453,"In April, during the Revolution of 1848 in Par...",What two people suggested the 1848 tour?,Jane Stirling and her elder sister,question: What two people suggested the 1848 t...,Jane Stirling and her elder sister
318,"The Battle of Long Island, the largest battle ...",In what borough did the Battle of Long Island ...,Brooklyn,question: In what borough did the Battle of Lo...,Brooklyn
